# 10. Calculation of Topological Network Metrics
One of the crucial steps of our analysis is clustering our cities into categories. In order to do so we leverage research from Geoff Boeing. In his seminal paper, Boeing identifies several key characteristics which help to distinguish urban environments based on characteristics. Since this study is interested in purely topological elements we leverage only the topological metric from Boeing's research.

This study is interested in comparing morphology to retail activity, as a result we cluster cities

In [9]:
# load relevant packages for analysis
import cityseer as cs
import contextily as ctx
import dask.dataframe as dd
import dask_geopandas as dg
import fiona
import graph_tool.all as gt
import geopandas as gpd
import igraph
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from matplotlib.collections import LineCollection
import numpy as np
import networkx as nx
import osmnx as ox
import pandas as pd
import pyogrio
import seaborn as sns
from scipy.stats import entropy
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler
from shapely.geometry import Point, MultiPoint
from shapely.geometry import LineString, MultiLineString
from shapely.ops import nearest_points
import shapely.wkt
from shapely.wkt import loads
import statsmodels.api as sm
import sys
from xml import etree

In [11]:
# G = nx.read_graphml('Data/london.graph')

In [16]:
G = nx.read_graphml("Data/G300_betw.graphml")

In [17]:
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 3161041
Number of edges: 3448220


In [18]:
list (G.nodes(data=True))[0]

('CC9BEF56-6F33-47ED-AF08-BB8A6C655A1D',
 {'pos': '(187422.39, 887423.74)',
  'form_of_road_node': 'junction',
  'geometry': 'POINT (187422.39 887423.74)',
  'angle': 126.44988586701946,
  'occupation_probability': 0.7024993659278859})

In [19]:
list(G.edges(data=True))[0]

('CC9BEF56-6F33-47ED-AF08-BB8A6C655A1D',
 'D3B5B123-54B7-467E-8C60-BE2383A3624B',
 {'length': 179.0,
  'road_classification': 'Unknown',
  'road_function': 'Restricted Local Access Road',
  'form_of_way': 'Single Carriageway',
  'primary_route': False,
  'trunk_road': False,
  'geometry': 'LINESTRING (187422.39 887423.74, 187436.12 887429.24, 187434.34 887455.56, 187439.76 887465.66, 187472.56 887482.08, 187552.42 887522.05)',
  'occupation_probability': 0.00927614779300641})

In [20]:
# Placeholder for your actual NetworkX graph
G_nx = G  # Your actual NetworkX graph
G_gt = gt.Graph(directed=False)

# Define vertex properties
pos = G_gt.new_vertex_property("vector<double>")
form_of_road_node = G_gt.new_vertex_property("string")
angle = G_gt.new_vertex_property("double")
occupation_probability_node = G_gt.new_vertex_property("double")
geometry = G_gt.new_vertex_property("string")

vertex_map = {}
node_count = 0

# Add a vertex property for original IDs
original_id = G_gt.new_vertex_property("string")
degree = G_gt.new_vertex_property("int")

# Add nodes to Graph-Tool graph
for node, data in G_nx.nodes(data=True):
    v = G_gt.add_vertex()
    vertex_map[node] = v
    pos[v] = list(map(float, data['pos'].strip('()').split(',')))
    form_of_road_node[v] = data.get('form_of_road_node', '')
    angle[v] = data.get('angle', 0.0)
    occupation_probability_node[v] = data.get('occupation_probability', 0.0)
    geometry[v] = data.get('geometry', '')
    original_id[v] = node  # Use the original node ID
    degree[v] = G_nx.degree(node)  # Assign degree of node
    node_count += 1

# Add the original_id property to the graph
G_gt.vertex_properties['original_id'] = original_id
G_gt.vertex_properties['pos'] = pos
G_gt.vertex_properties['form_of_road_node'] = form_of_road_node
G_gt.vertex_properties['angle'] = angle
G_gt.vertex_properties['occupation_probability_node'] = occupation_probability_node
G_gt.vertex_properties['geometry'] = geometry
G_gt.vertex_properties['degree'] = degree

print(f"Number of nodes added to Graph-Tool graph: {G_gt.num_vertices()}")
print(f"Number of nodes in vertex_map: {len(vertex_map)}")
print(f"Node count processed: {node_count}")

Number of nodes added to Graph-Tool graph: 3161041
Number of nodes in vertex_map: 3161041
Node count processed: 3161041


In [21]:
# Define edge properties
edge_length = G_gt.new_edge_property("double")
road_classification = G_gt.new_edge_property("string")
road_function = G_gt.new_edge_property("string")
form_of_way = G_gt.new_edge_property("string")
primary_route = G_gt.new_edge_property("bool")
trunk_road = G_gt.new_edge_property("bool")
edge_geometry = G_gt.new_edge_property("object")
occupation_probability_edge = G_gt.new_edge_property("double")
original_edge_id = G_gt.new_edge_property("string")

# Add edges to Graph-Tool graph
for u, v, data in G_nx.edges(data=True):
    e = G_gt.add_edge(vertex_map[u], vertex_map[v])
    edge_length[e] = data['length']
    road_classification[e] = data['road_classification']
    road_function[e] = data['road_function']
    form_of_way[e] = data['form_of_way']
    primary_route[e] = data['primary_route']
    trunk_road[e] = data['trunk_road']
    edge_geometry[e] = loads(data['geometry'])
    occupation_probability_edge[e] = data['occupation_probability']
    original_edge_id[e] = f"{u}-{v}"

# Add edge properties to the graph
G_gt.edge_properties['length'] = edge_length
G_gt.edge_properties['road_classification'] = road_classification
G_gt.edge_properties['road_function'] = road_function
G_gt.edge_properties['form_of_way'] = form_of_way
G_gt.edge_properties['primary_route'] = primary_route
G_gt.edge_properties['trunk_road'] = trunk_road
G_gt.edge_properties['geometry'] = edge_geometry
G_gt.edge_properties['occupation_probability'] = occupation_probability_edge
G_gt.edge_properties['original_edge_id'] = original_edge_id

In [22]:
# Calculate Betweenness Centrality
vertex_betweenness, edge_betweenness = gt.betweenness(G_gt)
G_gt.vertex_properties['betweenness'] = vertex_betweenness
G_gt.edge_properties['betweenness'] = edge_betweenness

# Calculate PageRank
pagerank = gt.pagerank(G_gt)
G_gt.vertex_properties['pagerank'] = pagerank

# Calculate Self-loop Proportion
num_self_loops = sum(1 for e in G_gt.edges() if e.source() == e.target())
self_loop_proportion = num_self_loops / G_gt.num_edges()
G_gt.graph_properties['self_loop_proportion'] = G_gt.new_graph_property("double")
G_gt.graph_properties['self_loop_proportion'] = self_loop_proportion

print("Topological metrics calculated and added to the graph.")

Topological metrics calculated and added to the graph.


In [34]:
def convert_gt_to_nx(G_gt):
    # Create a new NetworkX graph (directed or undirected)
    G_nx = nx.DiGraph() if G_gt.is_directed() else nx.Graph()

    # Check if 'original_id' property exists
    original_id_exists = 'original_id' in G_gt.vertex_properties

    # Transfer node properties
    for v in G_gt.vertices():
        node_data = {}
        for prop_name, prop in G_gt.vertex_properties.items():
            value = prop[v]
            if isinstance(value, gt.libgraph_tool_core.Vector_double):
                value = float(value[0]) if len(value) > 0 else None
            node_data[prop_name] = value
        # Use the original ID if it exists, otherwise use the default index
        if original_id_exists:
            original_id = G_gt.vertex_properties['original_id'][v]
        else:
            original_id = int(v)
        G_nx.add_node(original_id, **node_data)

    # Transfer edge properties
    for e in G_gt.edges():
        edge_data = {}
        for prop_name, prop in G_gt.edge_properties.items():
            value = prop[e]
            if isinstance(value, gt.libgraph_tool_core.Vector_double):
                value = float(value[0]) if len(value) > 0 else None
            elif isinstance(value, LineString):
                value = value.wkt
            edge_data[prop_name] = value
        # Preserve the original edge source and target IDs if they exist
        if original_id_exists:
            original_source = G_gt.vertex_properties['original_id'][e.source()]
            original_target = G_gt.vertex_properties['original_id'][e.target()]
        else:
            original_source = int(e.source())
            original_target = int(e.target())
        G_nx.add_edge(original_source, original_target, **edge_data)

    return G_nx

# Example usage
G_nx_converted = convert_gt_to_nx(G_gt)

In [35]:
# Verify the conversion by checking some properties
print(f"Number of nodes in converted NetworkX graph: {G_nx_converted.number_of_nodes()}")
print(f"Number of edges in converted NetworkX graph: {G_nx_converted.number_of_edges()}")

# Print an example node and its properties
if G_nx_converted.number_of_nodes() > 0:
    example_node = list(G_nx_converted.nodes(data=True))[0]
    print("Example Node in NetworkX graph:")
    print(example_node)

# Print an example edge and its properties
if G_nx_converted.number_of_edges() > 0:
    example_edge = list(G_nx_converted.edges(data=True))[0]
    print("Example Edge in NetworkX graph:")
    print(example_edge)

Number of nodes in converted NetworkX graph: 3161041
Number of edges in converted NetworkX graph: 3448220
Example Node in NetworkX graph:
('CC9BEF56-6F33-47ED-AF08-BB8A6C655A1D', {'original_id': 'CC9BEF56-6F33-47ED-AF08-BB8A6C655A1D', 'pos': 187422.39, 'form_of_road_node': 'junction', 'angle': 126.44988586701946, 'occupation_probability_node': 0.7024993659278859, 'geometry': 'POINT (187422.39 887423.74)', 'degree': 2, 'betweenness': 2.0015670828009153e-13, 'pagerank': 4.6169985436708254e-07})
Example Edge in NetworkX graph:
('CC9BEF56-6F33-47ED-AF08-BB8A6C655A1D', 'D3B5B123-54B7-467E-8C60-BE2383A3624B', {'length': 179.0, 'road_classification': 'Unknown', 'road_function': 'Restricted Local Access Road', 'form_of_way': 'Single Carriageway', 'primary_route': 0, 'trunk_road': 0, 'geometry': 'LINESTRING (187422.39 887423.74, 187436.12 887429.24, 187434.34 887455.56, 187439.76 887465.66, 187472.56 887482.08, 187552.42 887522.05)', 'occupation_probability': 0.00927614779300641, 'original_edge

In [36]:
# Ensure scalar attributes while preserving geometry and original names
def ensure_scalar_attributes_with_geometry_and_names(G_nx):
    for node, data in G_nx.nodes(data=True):
        for key, value in data.items():
            if isinstance(value, (list, np.ndarray)) and key != 'geometry':
                G_nx.nodes[node][key] = value[0] if len(value) > 0 else None
            elif isinstance(value, gt.libgraph_tool_core.Vector_double):
                G_nx.nodes[node][key] = float(value[0]) if len(value) > 0 else None

    for u, v, data in G_nx.edges(data=True):
        for key, value in data.items():
            if isinstance(value, (list, np.ndarray)) and key != 'geometry':
                G_nx.edges[u, v][key] = value[0] if len(value) > 0 else None
            elif isinstance(value, gt.libgraph_tool_core.Vector_double):
                G_nx.edges[u, v][key] = float(value[0]) if len(value) > 0 else None
            elif isinstance(value, LineString):
                G_nx.edges[u, v][key] = value.wkt

    for key, value in G_nx.graph.items():
        if isinstance(value, (list, np.ndarray)):
            G_nx.graph[key] = value[0] if len(value) > 0 else None
        elif isinstance(value, gt.libgraph_tool_core.Vector_double):
            G_nx.graph[key] = float(value[0]) if len(value) > 0 else None

# Ensure scalar attributes
ensure_scalar_attributes_with_geometry_and_names(G_nx_converted)

# Save the NetworkX graph as a GraphML file
nx.write_graphml(G_nx_converted, "Data/G_Globalattributes.graphml")

In [37]:
print(f"Number of nodes: {G_nx_converted.number_of_nodes()}")
print(f"Number of edges: {G_nx_converted.number_of_edges()}")

Number of nodes: 3161041
Number of edges: 3448220


In [38]:
list (G_nx_converted.nodes(data=True))[10]

('34788D06-0F04-4460-9F3C-AC7D297905B0',
 {'original_id': '34788D06-0F04-4460-9F3C-AC7D297905B0',
  'pos': 180425.0,
  'form_of_road_node': 'junction',
  'angle': 176.22704866155834,
  'occupation_probability_node': 0.9790391592308797,
  'geometry': 'POINT (180425 828365)',
  'degree': 3,
  'betweenness': 1.1008618955405034e-11,
  'pagerank': 4.777984313010437e-07})

In [39]:
list (G_nx_converted.edges(data=True))[0]

('CC9BEF56-6F33-47ED-AF08-BB8A6C655A1D',
 'D3B5B123-54B7-467E-8C60-BE2383A3624B',
 {'length': 179.0,
  'road_classification': 'Unknown',
  'road_function': 'Restricted Local Access Road',
  'form_of_way': 'Single Carriageway',
  'primary_route': 0,
  'trunk_road': 0,
  'geometry': 'LINESTRING (187422.39 887423.74, 187436.12 887429.24, 187434.34 887455.56, 187439.76 887465.66, 187472.56 887482.08, 187552.42 887522.05)',
  'occupation_probability': 0.00927614779300641,
  'original_edge_id': 'CC9BEF56-6F33-47ED-AF08-BB8A6C655A1D-D3B5B123-54B7-467E-8C60-BE2383A3624B',
  'betweenness': 4.0031316328069916e-13})